# Schlögl model

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate
from scripts.notebooks.custom_ssa import *

plt.style.use("./scripts/notebooks/custom_style.mplstyle")
%matplotlib inline

## Reaction system

In [ ]:
from scripts.reaction_class import Reaction, ReactionSystem

k1 = 2.5e-4
k2 = 0.18
k3 = 37.5
k4 = 2200

reactions = [None] * 4
reactions[0] = Reaction({0: lambda x: k1 * x * (x - 1) * (x - 2)}, [-1])
reactions[1] = Reaction({0: lambda x: k2 * x * (x - 1)}, [1])
reactions[2] = Reaction({0: lambda x: k3 * x}, [-1])
reactions[3] = Reaction({0: lambda x: k4}, [1])
species_names = ["A"]
reaction_system = ReactionSystem(reactions, species_names)

## Deterministic ODEs

In [ ]:
rhs = lambda t, y: (-k1 * y**3 + k2 * y**2 - k3 * y + k4)

t_span = (0, 1)
t_eval = np.linspace(0, 1, 100)
sol0 = scipy.integrate.solve_ivp(rhs, t_span, [0.0], t_eval=t_eval)
sol200 = scipy.integrate.solve_ivp(rhs, t_span, [200.0], t_eval=t_eval)
sol300 = scipy.integrate.solve_ivp(rhs, t_span, [300.0], t_eval=t_eval)
sol500 = scipy.integrate.solve_ivp(rhs, t_span, [500.0], t_eval=t_eval)

## Stochastic Simulation Algorithm

In [ ]:
initial_state = np.array([0])
sample_times = np.linspace(0, 100, 1000, endpoint=True)
trajectory = runSimulation(sample_times, 110.0, initial_state, 1, reaction_system)

In [ ]:
t_span_100 = (0, 100)
sol0_100 = scipy.integrate.solve_ivp(rhs, t_span_100, [0.0])

## Stationary probability distribution

In [ ]:
def phi(n):
    result = 1
    for i in range(n):
        result *= (k2 * i * (i - 1) + k4) / (k1 * (i + 1) * i * (i - 1) + k3 * (i + 1))
    return result

In [ ]:
gs = gridspec.GridSpec(2, 4)

fig = plt.figure(figsize=(6, 6))

ax1 = plt.subplot(gs[0, :])
ax1.plot(t_eval, 100 * np.ones(t_eval.size), 'k:')
ax1.plot(t_eval[:72], 220 * np.ones(t_eval.size)[:72], 'k:')
ax1.plot(t_eval, 400 * np.ones(t_eval.size), 'k:')
ax1.plot(sol0.t, sol0.y[0], label="$x(0)=0$")
ax1.plot(sol200.t, sol200.y[0], label="$x(0)=200$")
ax1.plot(sol300.t, sol300.y[0], label="$x(0)=300$")
ax1.plot(sol500.t, sol500.y[0], label="$x(0)=500$")

ax1.set_xlabel("$t$")
ax1.set_ylabel("$x(t)$")
ax1.legend()

ax2 = plt.subplot(gs[1, :3])
ax2.plot(sample_times, trajectory[:, :, 0].T, label="SSA");
ax2.plot(sol0_100.t, sol0_100.y[0], label="ODE")
ax2.set_xlabel("$t$")
ax2.set_ylabel("$x(t)$")
ax2.legend()

n = np.arange(600+1)
phi_tot = np.array([phi(i) for i in n])
phi_tot /= np.sum(phi_tot)
mean = phi_tot @ n
print(mean)
ax3 = plt.subplot(gs[1, 3], sharey=ax2)
ax3.plot(phi_tot, n)
ax3.plot([0, np.max(phi_tot)], mean * np.ones(2), "k:")
ax3.annotate("$\langle x \\rangle$",
            xy=(0.85, 0.225), xycoords='figure fraction',
            fontsize=12)
ax3.set_xlabel("$\Phi(x)$")
ax3.tick_params('y', labelleft=False)

plt.subplots_adjust(hspace=0.3, wspace=0.1);
plt.savefig("plots/schloegl.pdf")